In [ ]:
!pip install transformers
!pip install -U -q PyDrive

In [ ]:
!pip install simpletransformers
!pip freeze | grep simpletransformers

English Data

In [41]:
import pandas as pd
df=pd.read_csv('/content/eng_3_train.tsv', delimiter='\t') 
df=df[["text", "category"]]
df = df.dropna()
df_dev=pd.read_csv('/content/eng_3_dev.tsv', delimiter='\t') 
df_dev=df_dev[["text", "category"]]
df_dev = df_dev.dropna()
df_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'text'}) 
df_test = df_test.dropna()

In [42]:
tedf=pd.read_csv('/content/tam_eng_train.tsv', delimiter='\t') 
tedf=tedf[["text", "category"]]
tedf = tedf.dropna()
tedf_dev=pd.read_csv('/content/tam_eng_dev.tsv', delimiter='\t') 
tedf_dev=tedf_dev[["text", "category"]]
tedf_dev = tedf_dev.dropna()
tedf_test=pd.read_csv('/content/tam_eng_test.tsv', delimiter='\t') 
tedf_test = tedf_test.dropna()

### Mixing english and tamil translated to english data

In [43]:
train = pd.concat([df, tedf])
valid = pd.concat([df_dev,tedf_dev])
test = pd.concat([tedf_test,df_test])

In [44]:
print(len(train))
print(len(valid))
print(len(test))

5817
1456
1819


In [45]:
concatenated = pd.concat([train, test,valid])

import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

df = concatenated

X = df['text']
y = df['category']

testsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)

train_val_idx, test_idx = next(testsplit.split(X, y))

X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]


validsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)

train_idx, val_idx = next(validsplit.split(X_train_val, y_train_val))

X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]

train_df = pd.DataFrame({'text': X_train, 'category': y_train})
val_df = pd.DataFrame({'text': X_val, 'category': y_val})
test_df = pd.DataFrame({'text': X_test, 'category': y_test})

print(f'Training set shape: {train_df.shape}')
print(f'Validation set shape: {val_df.shape}')
print(f'Test set shape: {test_df.shape}')


Training set shape: (5454, 2)
Validation set shape: (1819, 2)
Test set shape: (1819, 2)


In [46]:
print(train_df['category'].value_counts())
print(val_df['category'].value_counts())
print(test_df['category'].value_counts())

Non-anti-LGBT+ content    4711
Homophobic                 598
Transphobic                145
Name: category, dtype: int64
Non-anti-LGBT+ content    1571
Homophobic                 199
Transphobic                 49
Name: category, dtype: int64
Non-anti-LGBT+ content    1571
Homophobic                 200
Transphobic                 48
Name: category, dtype: int64


In [47]:
train_df['category'] = train_df['category'].replace('Non-anti-LGBT+ content', 'ally')
val_df['category'] = val_df['category'].replace('Non-anti-LGBT+ content', 'ally')
test_df['category'] = test_df['category'].replace('Non-anti-LGBT+ content', 'ally')

In [48]:
train_df['category'] = train_df['category'].replace('ally', 0)
train_df['category'] = train_df['category'].replace('Homophobic', 1)
train_df['category'] = train_df['category'].replace('Transphobic', 2)

In [49]:
val_df['category'] = val_df['category'].replace('ally', 0)
val_df['category'] = val_df['category'].replace('Homophobic', 1)
val_df['category'] = val_df['category'].replace('Transphobic', 2)

In [50]:
test_df['category'] = test_df['category'].replace('ally', 0)
test_df['category'] = test_df['category'].replace('Homophobic', 1)
test_df['category'] = test_df['category'].replace('Transphobic', 2)

In [51]:
train_df=train_df[["text", "category"]]
train_df = train_df.rename(columns={'text': 'text', 'category': 'labels'})
val_df=val_df[["text", "category"]]
val_df = val_df.rename(columns={'text': 'text', 'category': 'labels'})
test_df=test_df[["text", "category"]]
test_df = test_df.rename(columns={'text': 'text', 'category': 'labels'})

In [ ]:
!pip install emoji

In [52]:
import emoji
train_df['text'] = train_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
val_df['text'] = val_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
test_df['text'] = test_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

## Training Model with Englis Data

In [27]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 3,
             "max_seq_length": 128,
             "train_batch_size": 32,
             "eval_batch_size": 32,
             "logging_steps": 50,
             "save_steps": 2000,
             "learning_rate": 2e-5,
             "manual_seed": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
    num_labels=3,
    args=train_args
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [28]:
# Train the model
model.train_model(train_df)

  0%|          | 0/5454 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/171 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/171 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/171 [00:00<?, ?it/s]

(513, 0.29298959750152004)

Validation Set


In [29]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')
result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score)
result

  0%|          | 0/1819 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/57 [00:00<?, ?it/s]

{'mcc': 0.603005958864853,
 'f1': 0.7212432097340188,
 'acc': 0.903243540406817,
 'eval_loss': 0.285741789462535}

Testing With English Data

In [53]:
test_df

,text,labels
547,How u will decide the nature,0
2903,"Interviewer very worse communication, you are ...",0
49,Androgen insentivity syndrome. Females who loo...,0
1480,You are from He'll. ... Let God intervene and ...,0
1191,All transgenders,0
...,...,...
1395,Only the Holy Word of God ...will be written i...,0
1371,Thanks for supporting sir,0
326,"A year ago, when my stepdad was there, he crie...",0
412,bible is unscientific. proven in Supreme Court...,0


In [54]:
test = test_df['text'].tolist()
labels = test_df['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,Does anyone forbid wage labor?,0,1
1,It is not something that occurs naturally. Art...,1,0
2,What is nature? The Supreme Court of India wil...,0,1
3,Bro it is very bad,1,0
4,All the pundits who come here and support this...,1,2
...,...,...,...
168,Why are you interviewing this dog?,0,2
169,"Sathya Narayanan , it was encouraged in wester...",1,0
170,Raping is also an individual choice only is th...,0,1
171,Unna patha le vandi vartu,1,0


In [33]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.95      0.95      0.95      1571
  homophobic       0.63      0.60      0.62       200
 transphobic       0.58      0.52      0.55        48

    accuracy                           0.90      1819
   macro avg       0.72      0.69      0.71      1819
weighted avg       0.90      0.90      0.90      1819



# Training with english and testing with tamil

In [100]:
o_test=pd.read_csv('/content/eng_tam_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_3_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [101]:
test_ml = pd.concat([o_test,e_test])

In [103]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [104]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [105]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [106]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,நீங்கள் மிகவும் சுத்தமாகப் பேசுபவர்... உடல் சே...,0,1
1,இது முற்றிலும் உயிரியல் தான்!,0,1
2,எனது கருத்துக்கு பதிலளித்த நீங்கள் இருவரும் இத...,1,0
3,"சத்திய நாராயணன் , இன்றும் மேற்கத்திய நாடுகளில்...",1,0
4,மனநோய் என்று நினைக்கிறேன். சென்று மருத்துவரை அ...,1,0
...,...,...,...
255,ரொம்ப நல்லவபோல நடிக்கறாள். பேசுகிறாள் காசுகொடு...,2,0
256,உங்கள் கருத்தை நான் ஏற்று கொள்கிறேன். ஓரினச் ச...,1,0
257,இந்த மனசு தான் sir கடவுள்....,0,1
258,அருமை பேச்சு .இனிமையான குரல். கேள்விக்கேற்ற பத...,1,0


In [107]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.89      0.96      0.92      1579
  homophobic       0.39      0.19      0.26       194
 transphobic       0.29      0.04      0.08        46

    accuracy                           0.86      1819
   macro avg       0.52      0.40      0.42      1819
weighted avg       0.82      0.86      0.83      1819



# Training with english and testing with Hindi

In [108]:
o_test=pd.read_csv('/content/eng_hin_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_hin_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [110]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [111]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [112]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [113]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [114]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,यह विशुद्ध रूप से जैविक है!,0,1
1,"आप दोनों जिन्होंने मेरी टिप्पणी का जवाब दिया, ...",1,0
2,जो व्यक्ति पूछ रहा है वह समलैंगिकों के बारे मे...,0,1
3,"सत्य नारायणन , इसे आज भी पश्चिमी दुनिया में प्...",1,0
4,समलैंगिक का क्या मतलब है,0,1
...,...,...,...
256,किसी लड़के को लड़का नहीं होना चाहिए,0,1
257,ये मन है साहब भगवान….,0,1
258,ऐसा झूठ क्यों है.. क्या वो साड़ी कहीं दिखा सकत...,0,1
259,समझें कि जानवरों में भी आत्म-जागरूकता होती है।...,0,1


In [115]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.92      0.93      0.92      1579
  homophobic       0.43      0.44      0.44       194
 transphobic       0.31      0.22      0.26        46

    accuracy                           0.86      1819
   macro avg       0.56      0.53      0.54      1819
weighted avg       0.85      0.86      0.85      1819



# Training with english and testing with Telugu

In [116]:
o_test=pd.read_csv('/content/eng_tel_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_tel_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [117]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [118]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [119]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [120]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [121]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,ఇది పూర్తిగా జీవసంబంధమైనది!,0,1
1,నా వ్యాఖ్యకు ప్రత్యుత్తరం ఇచ్చిన మీరిద్దరూ దీన...,1,0
2,"సత్య నారాయణన్ , నేటికీ పాశ్చాత్య ప్రపంచంలో దీన...",1,0
3,నా అభిప్రాయం .. ఇది మానసిక వ్యాధి. వెళ్లి వైద్...,1,0
4,ఇది చట్టబద్ధం చేయబడినందున అది సరైనదని అర్థం కాదు,0,1
...,...,...,...
264,"అనాథలు మీలాగా లెస్బియన్‌గా, గేలుగా ఎందుకు పెరు...",1,0
265,మీరు తెలివిగా మాట్లాడాలని అనుకుంటున్నారు...,1,0
266,ఆమె చాలా చక్కగా నటిస్తుంది. మంచూర్‌లో డబ్బులు ...,2,0
267,నేను మీ అభిప్రాయాన్ని అంగీకరిస్తున్నాను. స్వలి...,1,0


In [122]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.89      0.95      0.92      1579
  homophobic       0.38      0.21      0.27       194
 transphobic       0.32      0.17      0.23        46

    accuracy                           0.85      1819
   macro avg       0.53      0.45      0.47      1819
weighted avg       0.82      0.85      0.83      1819



# Training with english and testing with Malyalam

In [123]:
o_test=pd.read_csv('/content/eng_ml_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_ml_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [124]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [125]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [126]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [127]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [128]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,അയ്യോ അസൂയ???? അപ്പോൾ നിങ്ങൾ സ്വയം രാജ്ഞി എന്ന...,0,1
1,എല്ലാ ആശംസകളും.... പുഞ്ചിരിക്കുന്ന_മുഖം_3_ഹൃദയ...,0,1
2,"അതായത്...ഒരു ശ്രമം ഉണ്ടായോ?? അതേസമയം, സുന്ദരമാ...",0,1
3,ഇത് തികച്ചും ജൈവികമാണ്!,0,1
4,എന്റെ അഭിപ്രായത്തിന് മറുപടി നൽകിയ നിങ്ങൾ രണ്ടു...,1,0
...,...,...,...
297,താങ്കളുടെ അഭിപ്രായം ഞാൻ അംഗീകരിക്കുന്നു. സ്വവർ...,1,0
298,ഈ മനസ്സാണ് ദൈവമേ...,0,1
299,ഈ പെൺകുട്ടി തിരുവണ്ണാമലയാണ്,0,2
300,അതുകൊണ്ടാണ് നിങ്ങളുടെ ദൈവം നിങ്ങൾക്ക് ഈ വികാരം...,0,1


In [129]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.90      0.92      0.91      1579
  homophobic       0.30      0.27      0.29       194
 transphobic       0.48      0.28      0.36        46

    accuracy                           0.83      1819
   macro avg       0.56      0.49      0.52      1819
weighted avg       0.82      0.83      0.83      1819

